In [32]:
import java.nio.file.Paths
import org.apache.spark.sql.functions._
val dir_ot = Paths.get(System.getProperty("user.home"), "data", "ot", "extract")
var df = spark.read.json(dir_ot.resolve("evidence.json").toString)

dir_ot = /home/eczech/data/ot/extract
df = [access_level: string, disease: struct<biosample: struct<id: string, name: string>, efo_info: struct<efo_id: string, label: string ... 2 more fields> ... 3 more fields> ... 12 more fields]


[access_level: string, disease: struct<biosample: struct<id: string, name: string>, efo_info: struct<efo_id: string, label: string ... 2 more fields> ... 3 more fields> ... 12 more fields]

In [33]:
df.select("scores").printSchema()

root
 |-- scores: struct (nullable = true)
 |    |-- association_score: double (nullable = true)



In [35]:
// ES Query:
// GET master_evidence-data/_search
// {
//   "query": {
//     "bool": {
//       "must": [
//         { "match": { "disease.id": "EFO_0001645" }},
//         { "match": { "target.id": "ENSG00000226777"}},
//         { "match": { "sourceID": "sysbio"}}
//       ]
//     }
//   }
// }

df.filter($"id" === "abb040109d9414a1f5e764f031ffc3b4")
    .select($"scores", $"target.id", $"disease.id", $"sourceID").show()

+------+---------------+-----------+--------+
|scores|             id|         id|sourceID|
+------+---------------+-----------+--------+
| [1.0]|ENSG00000226777|EFO_0001645|  sysbio|
+------+---------------+-----------+--------+



In [5]:
df.groupBy("sourceID").count().show()

+------------------+------+
|          sourceID| count|
+------------------+------+
|            crispr|  1659|
|  genomics_england| 10528|
|            chembl|383122|
|        slapenrich| 74570|
|      gwas_catalog|180984|
|  expression_atlas|204229|
|           intogen|  2375|
|           uniprot| 30480|
|          reactome| 10159|
|   uniprot_somatic|   284|
|            sysbio|   408|
|         phenodigm|500683|
|    phewas_catalog| 55987|
|cancer_gene_census| 60310|
|           progeny|   308|
|               eva| 96734|
|    gene2phenotype|  1586|
|uniprot_literature|  4553|
|       eva_somatic|  7585|
+------------------+------+



In [15]:
df.filter($"sourceID" === "sysbio").show(3)

+------------+--------------------+----+--------------------+--------------------+----------+--------------------+--------+--------+--------------------+----------------+-------------------------+--------------------------------+-------+
|access_level|             disease|drug|            evidence|                  id|literature|             private|  scores|sourceID|              target|            type|unique_association_fields|validated_against_schema_version|variant|
+------------+--------------------+----+--------------------+--------------------+----------+--------------------+--------+--------+--------------------+----------------+-------------------------+--------------------------------+-------+
|      public|[, [http://www.eb...|null|[,,,, 2019-01-31T...|81d85d92cd5a1fb27...|      null|[sysbio, affected...|[0.7844]|  sysbio|[unknown,,, [ENSG...|affected_pathway|     [,,,,,, sysbio,,,...|                           1.2.8|   null|
|      public|[, [http://www.eb...|null|[,,,, 20

In [25]:
var dfe = df
    .filter($"disease.id" === "EFO_0001645")
    .filter($"target.id" === "ENSG00000226777")
    .filter($"sourceID" === "sysbio")
    
dfe.show(3)

+------------+--------------------+----+--------------------+--------------------+----------+--------------------+-------+--------+--------------------+----------------+-------------------------+--------------------------------+-------+
|access_level|             disease|drug|            evidence|                  id|literature|             private| scores|sourceID|              target|            type|unique_association_fields|validated_against_schema_version|variant|
+------------+--------------------+----+--------------------+--------------------+----------+--------------------+-------+--------+--------------------+----------------+-------------------------+--------------------------------+-------+
|      public|[, [http://www.eb...|null|[,,,, 2019-01-31T...|abb040109d9414a1f...|      null|[sysbio, affected...|[0.859]|  sysbio|[unknown,,, [ENSG...|affected_pathway|     [,,,,,, sysbio,,,...|                           1.2.8|   null|
+------------+--------------------+----+------------

dfe = [access_level: string, disease: struct<biosample: struct<id: string, name: string>, efo_info: struct<efo_id: string, label: string ... 2 more fields> ... 3 more fields> ... 12 more fields]


[access_level: string, disease: struct<biosample: struct<id: string, name: string>, efo_info: struct<efo_id: string, label: string ... 2 more fields> ... 3 more fields> ... 12 more fields]

Results from scoring-poc after explosion:

```
+--------------------+---------+---------------+---------------+-----+-----------+------------+
|                  id|source_id|orig_disease_id|      target_id|score| disease_id|is_direct_id|
+--------------------+---------+---------------+---------------+-----+-----------+------------+
|abb040109d9414a1f...|   sysbio|    EFO_0001645|ENSG00000226777|0.859|EFO_0003777|       false|
|abb040109d9414a1f...|   sysbio|    EFO_0001645|ENSG00000226777|0.859|EFO_0000408|       false|
|abb040109d9414a1f...|   sysbio|    EFO_0001645|ENSG00000226777|0.859|EFO_0000319|       false|
|abb040109d9414a1f...|   sysbio|    EFO_0001645|ENSG00000226777|0.859|EFO_0001645|        true|
+--------------------+---------+---------------+---------------+-----+-----------+------------+
```

In [26]:
dfe.select($"scores", $"target", $"disease", $"disease.id").show(1, 1000)

+-------+-----------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+
| scores|                                                                                                                       target|                                                                                                                                                                   disease|         id|
+-------+-----------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+
|[0.859]|[unknown,,, [ENSG00000226777, fami

In [3]:
df.printSchema

root
 |-- access_level: string (nullable = true)
 |-- disease: struct (nullable = true)
 |    |-- biosample: struct (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |-- efo_info: struct (nullable = true)
 |    |    |-- efo_id: string (nullable = true)
 |    |    |-- label: string (nullable = true)
 |    |    |-- path: array (nullable = true)
 |    |    |    |-- element: array (containsNull = true)
 |    |    |    |    |-- element: string (containsNull = true)
 |    |    |-- therapeutic_area: struct (nullable = true)
 |    |    |    |-- codes: array (nullable = true)
 |    |    |    |    |-- element: string (containsNull = true)
 |    |    |    |-- labels: array (nullable = true)
 |    |    |    |    |-- element: string (containsNull = true)
 |    |-- id: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- source_name: string (nullable = true)
 |-- drug: struct (nullable = true)
 |    |-- id: string